In [24]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [26]:
notes = []

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try:
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

Parsing midi_songs\0fithos.mid
Parsing midi_songs\8.mid
Parsing midi_songs\ahead_on_our_way_piano.mid
Parsing midi_songs\AT.mid
Parsing midi_songs\balamb.mid
Parsing midi_songs\bcm.mid
Parsing midi_songs\BlueStone_LastDungeon.mid
Parsing midi_songs\braska.mid
Parsing midi_songs\caitsith.mid
Parsing midi_songs\Cids.mid
Parsing midi_songs\cosmo.mid
Parsing midi_songs\costadsol.mid
Parsing midi_songs\dayafter.mid
Parsing midi_songs\decisive.mid
Parsing midi_songs\dontbeafraid.mid
Parsing midi_songs\DOS.mid
Parsing midi_songs\electric_de_chocobo.mid
Parsing midi_songs\Eternal_Harvest.mid
Parsing midi_songs\EyesOnMePiano.mid
Parsing midi_songs\ff11_awakening_piano.mid
Parsing midi_songs\ff1battp.mid
Parsing midi_songs\FF3_Battle_(Piano).mid
Parsing midi_songs\FF3_Third_Phase_Final_(Piano).mid
Parsing midi_songs\ff4-airship.mid
Parsing midi_songs\Ff4-BattleLust.mid
Parsing midi_songs\ff4-fight1.mid
Parsing midi_songs\ff4-town.mid
Parsing midi_songs\FF4.mid
Parsing midi_songs\ff4pclov.mid
Par

In [27]:
n_vocab = len(set(notes))
n_vocab

326

In [30]:
sequence_length = 100

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [31]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3, ))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [32]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

In [ ]:
model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)